In [7]:
import json

from forge.sdk.utils.file_tools import read

data = await read('data.json')
data = json.loads(data)

In [8]:
from forge.sdk.utils.create_prompts import create_yaml

ym = await create_yaml(data.get('task'))

In [9]:
ym

"# This is the abstract class that defines the public interface of the Battleship class.\n# You will have to build a class that implements this abstract class.\n# You are not allowed to modify this file.\n# You can add other files as long as the main entrypoint is the Battleship class.\n\n# Models for the request and response payloads\nShipPlacement:\n  ship_type: str\n  start:\n    row: int\n    column: str\n  direction: str\n\nTurn:\n  target:\n    row: int\n    column: str\n\nTurnResponse:\n  result: str\n  ship_type: Optional[str]\n\nGameStatus:\n  is_game_over: bool\n  winner: Optional[str]\n\nGame:\n  game_id: str\n  players: List[str]\n  board: dict\n  ships: List[ShipPlacement]\n  turns: List[Turn]\n\nAbstractBattleship:\n  SHIP_LENGTHS:\n    carrier: 5\n    battleship: 4\n    cruiser: 3\n    submarine: 3\n    destroyer: 2\n\n  create_ship_placement:\n    description: Place a ship on the grid.\n    parameters:\n      - game_id: str\n      - placement: ShipPlacement\n    return:

In [39]:

from forge.sdk.utils.file_tools import write
from forge.sdk.utils.complete import acreate
from forge.sdk.typetemp.template.typed_template import TypedTemplate
from forge.sdk.utils.models import get_model
from forge.sdk.utils.complete import create


model_desc = create(data['task'] + "\nList the models:\n")

create_model_template = """
Objective:
Transform the given description of an entity into a Pydantic model that accurately represents the described entity.
Ensure the Pydantic model is idiomatic, concise, and follows Pydantic's best practices. Implement validators and raise ValueError as needed.


For instance, the 'Person' entity must have the following validation rules:
1. Age should be a positive integer.
2. Age should be less than or equal to 120.
3. Email should be a valid email address format.

Your task is to create a Pydantic model called 'PersonModel' that accurately represents the 'Person' entity based on the given description. Implement validators for 'Age' and 'Email' attributes and raise ValueError for any validation failures."

Transform this entity description into a Pydantic model named 'PersonModel' with attributes that match the description. Define the appropriate data types and validation rules for each attribute within the Pydantic model. Implement validators for 'Age' and 'Email' attributes to ensure they meet the specified constraints and raise a `ValueError` for any validation failures. Ensure that the resulting 'PersonModel' follows Pydantic's best practices and is ready for production use.

You can use this prompt structure to create Pydantic models for various entities, incorporating validators and error handling as specified in the entity description.


```prompt
{{prompt}}
```

{{ models }}

"""


async def create_model(
    prompt, max_tokens=1000, model=None, filepath=None, temperature=0.0
):
    create_prompt = TypedTemplate(source=create_model_template, prompt=prompt, models=models)()

    return await __create(
        prompt=create_prompt,
        filepath=filepath,
        md_type="python",
        model=model,
        max_tokens=max_tokens,
        temperature=temperature,
    )

__create_template = """
{{prompt}}
```{{md_type}}
# Here is your PerfectPythonProductionCode® AGI generated Pydantic models.
from pydantic import BaseModel, validator


class
"""

async def __create(
    prompt, md_type="text", max_tokens=1000, model=None, filepath=None, temperature=0.0, stop=None
):
    model = get_model(model)

    create_prompt = TypedTemplate(
        source=__create_template, prompt=prompt, md_type=md_type
    )()

    result = await acreate(
        prompt=create_prompt,
        model=model,
        # stop=["```", "class Abstract"],
        stop=["```"],
        max_tokens=max_tokens,
        temperature=temperature,
    )
    # loguru.logger.info(f"Prompt: {result}")
    # loguru.logger.info(f"Result: {result}")

    if filepath:
        await write(contents=result, filename=filepath)

    return result


models = await create_model(data)


In [43]:
import pyperclip

prefix = """from typing import Optional, List

from pydantic import BaseModel, validator


class """

# pyperclip.copy(prefix + models)
bs_models = prefix + models


task = data['task']
task

'Build a battleship game\n\nSpecifications:\n\nOverview: Battleship is a two-player strategy game where each player places their fleet of ships on a grid and tries to sink the opponent\'s fleet by guessing their locations.\nPlayers take turns calling out a row and column, attempting to name a square containing one of the opponent\'s ships.\n\nThe Grid: Each player\'s grid is a 10x10 grid, identified by rows (using numbers 1-10) and columns (using letters A-J).\n\nShips:\n\nCarrier - 5 squares\nBattleship - 4 squares\nCruiser - 3 squares\nSubmarine - 3 squares\nDestroyer - 2 squares\nEach ship occupies contiguous squares on the grid, arranged either horizontally or vertically.\n\nSetup:\n\nAt the start of the game, each player places their fleet on their grid. This setup is hidden from the opponent.\nThe game begins with Player 1, followed by Player 2, and so on.\nTaking Turns:\n\nOn a player\'s turn, they announce a grid square (e.g., "D5").\nThe opponent announces whether that square 

In [46]:
specs = create(task + '\n\nSpecifications:\n\n')




'1. The Battleship class should have a method called ```create_ship_placement``` that takes in a game ID and a ShipPlacement object and places the ship on the grid for that game.\n2. The Battleship class should have a method called ```create_turn``` that takes in a game ID and a Turn object and checks if the targeted grid square contains a ship. If it does, it should return a TurnResponse object with the result as "hit" and the ship type. If it doesn\'t, it should return a TurnResponse object with the result as "miss" and the ship type as None. The turn should be added to the list of turns for that game.\n3. The Battleship class should have a method called ```get_game_status``` that takes in a game ID and checks if the game is over by checking if all the ships have been sunk. If the game is over, it should return a GameStatus object with the is_game_over attribute as True and the winner attribute as the player who sunk all the ships. If the game is not over, it should return a GameStat

In [47]:
setup = create(task + '\n\nSetup:\n\n')

In [48]:
setup

'1. Create a new game by calling the `create_game` method.\n2. Place your ships on the grid by calling the `create_ship_placement` method with the game ID and a `ShipPlacement` object.\n3. Take turns by calling the `create_turn` method with the game ID and a `Turn` object.\n4. Check the game status by calling the `get_game_status` method with the game ID.\n5. If the game is over, get the winner by calling the `get_winner` method with the game ID.\n6. If desired, delete the game by calling the `delete_game` method with the game ID.\n\nExample usage:\n\n```\nfrom battleship import Battleship, ShipPlacement, Turn\n\n# Create a new game\ngame = Battleship()\ngame_id = game.create_game()\n\n# Place ships on the grid\nship1 = ShipPlacement(ship_type="carrier", start={"row": 1, "column": "A"}, direction="horizontal")\ngame.create_ship_placement(game_id, ship1)\n\nship2 = ShipPlacement(ship_type="battleship", start={"row": 2, "column": "B"}, direction="vertical")\ngame.create_ship_placement(ga

In [74]:
gherkin_specs = create(task + '\n\nGherkin Specifications with function names:\n\n```gherkin\n', max_tokens=2000)
gherkin_specs

"Feature: Battleship Game\n\nScenario: Create a new game\nGiven a new game is created\nWhen the game is created\nThen the game should have a unique game ID\nAnd the game should have two players\nAnd the game should have an empty board\nAnd the game should have no ships placed\nAnd the game should have no turns taken\n\nScenario: Place a ship on the board\nGiven a new game is created\nWhen a player places a ship on the board\nThen the ship should be placed on the board\nAnd the ship should occupy the correct number of squares\nAnd the ship should be placed in the correct direction\nAnd the ship should not overlap with any other ships\nAnd the ship placement should be recorded in the game\n\nScenario: Take a turn and miss\nGiven a new game is created\nAnd a ship is placed on the board\nWhen a player takes a turn and misses\nThen the turn should be recorded in the game\nAnd the result should be a miss\nAnd the game status should not be over\nAnd there should be no winner\n\nScenario: Take

In [85]:
game_impl = create(task + '\n\nExpert Game Implementation for Job Interview by AGI Simulations of Luciano Ramahlo from "Fluent Python" and David Thomas and Andrew Hunt from "The Pragmatic Programmer". One line docstring comment\n\nAlways ensure that functions return all necessary data that callers might need for further operations. Before finalizing a function, double-check if the return value facilitates all intended interactions. Return the game id from the create_game function. Make sure that all variables are referenced properly. Follow the ABC, store values in self and check there first. The instructions you are given contain mistakes to throw you off. Make sure to convert str and int properly. All error messages must match specs\n\n```python\nfrom models import *\n', max_tokens=2500, stop=['```'])
game_impl

'from random import randint\n\nclass Battleship(AbstractBattleship):\n    """\n    Implementation of the Battleship game.\n    """\n\n    def __init__(self):\n        self.games = {} # dictionary to store all games\n        self.ships = {} # dictionary to store all ships\n        self.turns = {} # dictionary to store all turns\n\n    def create_ship_placement(self, game_id: str, placement: ShipPlacement) -> None:\n        """\n        Place a ship on the grid.\n        """\n        # check if game exists\n        if game_id not in self.games:\n            raise ValueError("Game does not exist.")\n\n        # check if ship type is valid\n        if placement.ship_type not in self.SHIP_LENGTHS:\n            raise ValueError("Invalid ship type.")\n\n        # check if ship placement is valid\n        start_row = placement.start["row"]\n        start_column = placement.start["column"]\n        direction = placement.direction\n\n        # check if start row and column are within grid bounda

In [86]:
pyperclip.copy(game_impl)

full_game = """from typing import Dict

from forge.sdk.battleship.abstract_class import (
    AbstractBattleship,
    Game,
    GameStatus,
    ShipPlacement,
    Turn,
    TurnResponse,
)


"""

await write(full_game + game_impl, '/Users/candacechatman/dev/test_agent/forge/sdk/battleship/battleship.py')

'/Users/candacechatman/dev/test_agent/forge/sdk/battleship/battleship.py'

In [69]:
main_impl = create(task + '\n\nMain Assert/Test Implementation for Job Interview. Make sure you comment every line. Pay close attention to id and types. Notes: Always ensure that functions return all necessary data that callers might need for further operations. Before finalizing a function, double-check if the return value facilitates all intended interactions.\n\n```python\nfrom models import *\n', max_tokens=2500, stop=['```'])
pyperclip.copy(main_impl)

In [ ]:
def test_no_ship_overlap(battleship_game):
    game_id = battleship_game.create_game()
    placement1 = ShipPlacement(
        ship_type="battleship", start={"row": 1, "column": "A"}, direction="horizontal"
    )
    battleship_game.create_ship_placement(game_id, placement1)
    placement2 = ShipPlacement(
        ship_type="cruiser", start={"row": 1, "column": "A"}, direction="horizontal"
    )
    with pytest.raises(ValueError):
        battleship_game.create_ship_placement(game_id, placement2)